In [1]:
from bn_codes import *

**`Discretização:`**

We give a number, int or float, and it gives us the discretized value, in order to uso on de BN.

In [2]:
x = spindle_temp_bin(87)
print(x)

high


In [3]:
x = vibration_rms_bin(1.0)
print(x)

moderate


In [4]:
x = coolant_flow_bin(0.35)
print(x)

low


**`Load a BN model:`**

The different models should use different functions.

NOTE: The debug variable must be turned off when using the app.

**MLE Model:** Can only predict overheat.

In [5]:
model_mle = load_model("models/model_mle.pkl", check=True, debug=True)
print("MLE model loaded - status OK")

Model check passed.
MLE model loaded - status OK


**ME Model:** Can predict overheat and get the diagnosis, but has the latent values switched (ex: FanFault = 0 increases overheat probability, but in reality it should be FanFault = 1 that increases overheat, because we defined that latent = 1 implies fault)

In [6]:
model_em_loaded = load_model("models/model_em.pkl", check=True, debug=True)
print("EM model loaded - status OK")

Model check passed.
EM model loaded - status OK


**ME Model Corrected:** Can predict overheat and get the correct diagnosis. On this model we switched the meaning of latent 0 and 1, therefore we now get the correct diagnoses.

In [7]:
model_em_corrected_loaded = load_model("models/model_em_corrected.pkl", check=True, debug=True)
print("EM model corrected loaded - status OK")

Model check passed.
EM model corrected loaded - status OK


**ME Model with Init CPDs:** Can predict overheat and get the correct diagnoses.

In [8]:
model_em_2_loaded = load_model("models/model_em_2.pkl", check=True, debug=True)
print("EM model 2 loaded - status OK")

Model check passed.
EM model 2 loaded - status OK


**`Make Overheat Prediction on the MLE Model:`**

The function needs to receive a dictionary, which is the evidence, that is, we want to know the P(Overheat|evidence).

The evidence can be something like:
- {"SpindleTemp": "low", "Vibration": "high", "CoolantFlow": "low"}
- {"SpindleTemp": "high"}
- {"SpindleTemp": "low", "CoolantFlow": "low"}

It only returns the probability of overheat.

NOTE: if we change the overheat_state to 0, then we are calculating P(no overheat|evidence).

In [9]:
evidence_case = {"SpindleTemp": "high", "Vibration": "high", "CoolantFlow": "low"}
p_case = overheat_probability(model_mle, evidence_case)
print("\nSpecific Case:", evidence_case)
print(f"P(Overheat=1 | evidence) = {p_case:.4f}")


Specific Case: {'SpindleTemp': 'high', 'Vibration': 'high', 'CoolantFlow': 'low'}
P(Overheat=1 | evidence) = 0.9976


**`Make Overheat Prediction and get the diagnoses on the ME Models:`** 

This function needs to receive the same type of dictionary with the evidences.

It returns the overheat probability (P(overheat=1|evidence)) and returns a ranking of the most probable diagnoses (P(latent=fault|evidence, overheat=1)). 

The ranking is in the following format: list[(latent_name, p_fault)]

Note: This function also needs the overheat_probability function.

Note: The model_em_loaded will not give correct results, due to the label switching.

In [10]:
evidence_case = {"SpindleTemp": "high", "Vibration": "high", "CoolantFlow": "low"}
p_overheat, ranking = diagnose_overheat(model_em_2_loaded, evidence=evidence_case)

This is not necessary in the app, but here is the diagnosis of this specific case:

In [11]:
print_diagnosis(p_overheat, ranking, evidence_case)

Evidence (sensors): {'SpindleTemp': 'high', 'Vibration': 'high', 'CoolantFlow': 'low'}
P(Overheat=1 | evidence) = 0.9976

Ranking of latent causes (conditioned on Overheat=1):
  - BearingWear: P(fault) = 0.9499
  - FanFault: P(fault) = 0.7498
  - CoolingEfficiency: P(fault) = 0.6913
  - CloggedFilter: P(fault) = 0.5163
